In [1]:
# === STEP 1: Load Data ===
import pandas as pd

file_path = "Pubchem_data_SGLT2_03112025.xlsx"
sheet_names = pd.ExcelFile(file_path).sheet_names

all_data = []
for s in sheet_names:
    df = pd.read_excel(file_path, sheet_name=s)
    sub = df[["PUBCHEM_CID", "PUBCHEM_EXT_DATASOURCE_SMILES", "PUBCHEM_ACTIVITY_OUTCOME"]].copy()
    all_data.append(sub)

data = pd.concat(all_data, ignore_index=True)
data.columns = ["PubChemID", "SMILES", "Activity_Class"]
data.drop_duplicates(inplace=True)
data.dropna(subset=["SMILES", "Activity_Class"], inplace=True)

print(data.shape)
data.head()


(590, 3)


,PubChemID,SMILES,Activity_Class
0,46889407,C1=CC(=C(C=C1[C@H]2[C@@H]([C@H]([C@@H]([C@H](O...,Active
1,46889427,CCOC1=NN=C(C=C1)CC2=C(C=CC(=C2)[C@H]3[C@@H]([C...,Active
2,46889428,CCCOC1=NN=C(C=C1)CC2=C(C=CC(=C2)[C@H]3[C@@H]([...,Active
3,46889429,CC(C)OC1=NN=C(C=C1)CC2=C(C=CC(=C2)[C@H]3[C@@H]...,Active
4,46889430,CCCCOC1=NN=C(C=C1)CC2=C(C=CC(=C2)[C@H]3[C@@H](...,Active


In [2]:
# === STEP 2: Generate Morgan Fingerprints ===
from rdkit import Chem
from rdkit.Chem import AllChem
import numpy as np

def smiles_to_fp_bits(smiles, radius=2, n_bits=1024):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return np.zeros(n_bits, dtype=np.float64)
    fp = AllChem.GetMorganFingerprintAsBitVect(mol, radius, nBits=n_bits)
    arr = np.zeros((n_bits,), dtype=np.int8)
    Chem.DataStructs.ConvertToNumpyArray(fp, arr)
    return arr.astype(np.float64)

X = np.vstack([smiles_to_fp_bits(s) for s in data["SMILES"]])
y = (data["Activity_Class"].str.lower() == "active").astype(int).values

print("Feature matrix shape:", X.shape)
print("Target vector shape:", y.shape)


[07:33:58] DEPRECATION WARNING: please use MorganGenerator
[07:33:58] DEPRECATION WARNING: please use MorganGenerator
[07:33:58] DEPRECATION WARNING: please use MorganGenerator
[07:33:58] DEPRECATION WARNING: please use MorganGenerator
[07:33:58] DEPRECATION WARNING: please use MorganGenerator
[07:33:58] DEPRECATION WARNING: please use MorganGenerator
[07:33:58] DEPRECATION WARNING: please use MorganGenerator
[07:33:58] DEPRECATION WARNING: please use MorganGenerator
[07:33:58] DEPRECATION WARNING: please use MorganGenerator
[07:33:58] DEPRECATION WARNING: please use MorganGenerator
[07:33:58] DEPRECATION WARNING: please use MorganGenerator
[07:33:58] DEPRECATION WARNING: please use MorganGenerator
[07:33:58] DEPRECATION WARNING: please use MorganGenerator
[07:33:58] DEPRECATION WARNING: please use MorganGenerator
[07:33:58] DEPRECATION WARNING: please use MorganGenerator
[07:33:58] DEPRECATION WARNING: please use MorganGenerator
[07:33:58] DEPRECATION WARNING: please use MorganGenerat

Feature matrix shape: (590, 1024)
Target vector shape: (590,)


[07:33:58] DEPRECATION WARNING: please use MorganGenerator
[07:33:58] DEPRECATION WARNING: please use MorganGenerator
[07:33:58] DEPRECATION WARNING: please use MorganGenerator
[07:33:58] DEPRECATION WARNING: please use MorganGenerator
[07:33:58] DEPRECATION WARNING: please use MorganGenerator
[07:33:58] DEPRECATION WARNING: please use MorganGenerator
[07:33:58] DEPRECATION WARNING: please use MorganGenerator
[07:33:58] DEPRECATION WARNING: please use MorganGenerator
[07:33:58] DEPRECATION WARNING: please use MorganGenerator
[07:33:58] DEPRECATION WARNING: please use MorganGenerator
[07:33:58] DEPRECATION WARNING: please use MorganGenerator
[07:33:58] DEPRECATION WARNING: please use MorganGenerator
[07:33:58] DEPRECATION WARNING: please use MorganGenerator
[07:33:58] DEPRECATION WARNING: please use MorganGenerator
[07:33:58] DEPRECATION WARNING: please use MorganGenerator
[07:33:58] DEPRECATION WARNING: please use MorganGenerator
[07:33:58] DEPRECATION WARNING: please use MorganGenerat

In [3]:
# === Sanity checks & prune dead columns ===
import numpy as np

print("NaN in X:", np.isnan(X).sum(), " | Inf in X:", np.isinf(X).sum())
print("NaN in y:", np.isnan(y).sum(), " | Inf in y:", np.isinf(y).sum())

feature_sums = X.sum(axis=0)
zero_cols = (feature_sums == 0)
print(f"Total features: {X.shape[1]}")
print(f"Zero-variance (always 0) features: {zero_cols.sum()}")

if zero_cols.sum() > 0:
    X = X[:, ~zero_cols]
    print("New X shape:", X.shape)
else:
    print("No zero-variance features removed.")


NaN in X: 0  | Inf in X: 0
NaN in y: 0  | Inf in y: 0
Total features: 1024
Zero-variance (always 0) features: 168
New X shape: (590, 856)


In [ ]:
!conda install scikit-learn

In [ ]:
# === STEP 3: Split data ===
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y    
)

X_train.shape, X_val.shape, y_train.shape, y_val.shape
# Add intercept column
X = np.c_[np.ones(X.shape[0]), X]
print("Updated X shape with intercept:", X.shape)  # Should be (590, 857)


In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
# === STEP 4: Logistic Regression from scratch ===
import numpy as np, sys

def sigmoid(z):
    # numerically stable sigmoid
    z = np.clip(z, -40, 40)
    return 1.0 / (1.0 + np.exp(-z))

def compute_cost(X, y, theta, lambda_=0.0):
    m = y.shape[0]
    h = sigmoid(X @ theta)
    # add small eps for log stability
    eps = 1e-12
    cost = (-1/m) * (y @ np.log(h + eps) + (1 - y) @ np.log(1 - h + eps))
    reg = (lambda_ / (2*m)) * np.sum(theta[1:]**2)
    return cost + reg

def gradient_descent(X, y, theta, alpha=0.01, num_iters=200, lambda_=0.1, tol=1e-6, print_every=1):  # Start with 1 for debug
    m = y.shape[0]
    J_history = []
    prev = np.inf
    for i in range(num_iters):
        h = sigmoid(X @ theta)
        grad = (1/m) * (X.T @ (h - y))
        grad[1:] += (lambda_/m) * theta[1:]
        theta -= alpha * grad

        J = compute_cost(X, y, theta, lambda_)
        J_history.append(J)

        if (i % print_every) == 0:
            print("Iter {:4d} | Cost: {:.6f}".format(i, J))
            sys.stdout.flush()

        if abs(prev - J) < tol:
            print("Early stopping at iter {} | Cost stabilized: {:.6f}".format(i, J))
            break
        prev = J
    return theta, J_history


In [ ]:
import numpy as np
import matplotlib
from rdkit import __version__ as rdkit_ver
print("NumPy:", np.__version__)
print("Matplotlib:", matplotlib.__version__)
print("RDKit:", rdkit_ver)
print("X_train shape & any NaN/Inf:", X_train.shape, np.isnan(X_train).sum(), np.isinf(X_train).sum())
print("y_train mean (active fraction):", y_train.mean())

In [ ]:
import traceback

# Your setup (theta0 etc.)
theta0 = np.zeros(X_train.shape[1], dtype=np.float64)
alpha = 0.01
iters = 5
lambda_ = 0.1

try:
    print("Starting GD...")
    theta_bl, J_hist = gradient_descent(X_train, y_train, theta0, alpha=alpha, num_iters=iters, lambda_=lambda_, tol=1e-6, print_every=1)  # Verbose for debug
    print("GD finished! Costs:", J_hist)
except Exception as e:
    print("Error during GD:", e)
    traceback.print_exc()  # Full stack trace

In [ ]:
# === Train baseline model ===
theta0 = np.zeros(X_train.shape[1], dtype=np.float64)
alpha   = 0.01
iters   = 5
lambda_ = 0.1

theta_bl, J_hist = gradient_descent(X_train, y_train, theta0, alpha=alpha, num_iters=iters, lambda_=lambda_, tol=1e-6, print_every=25)

# import matplotlib.pyplot as plt
# plt.plot(J_hist)
# plt.xlabel("Iteration")
# plt.ylabel("Cost J(θ)")
# plt.title("Training Cost Convergence")
# plt.show()

print("Final training cost:", J_hist[-1])


In [ ]:
# === STEP 5: Lambda tuning ===
lambda_values = [0, 0.01, 0.1, 1, 10]
train_costs, val_costs, thetas = [], [], []

for lam in lambda_values:
    th0 = np.zeros(X_train.shape[1], dtype=np.float64)
    th, JH = gradient_descent(X_train, y_train, th0, alpha=0.01, num_iters=250, lambda_=lam, tol=1e-6, print_every=100)
    thetas.append(th)
    train_costs.append(compute_cost(X_train, y_train, th, lam))
    val_costs.append(compute_cost(X_val, y_val, th, lam))

import matplotlib.pyplot as plt
plt.figure(figsize=(7,5))
plt.plot(lambda_values, train_costs, marker='o', label='Train cost')
plt.plot(lambda_values, val_costs, marker='o', label='Validation cost')
plt.xscale('log'); plt.xlabel('λ'); plt.ylabel('Cost'); plt.title('Regularization Tuning'); plt.legend(); plt.show()

for lam, tr, va in zip(lambda_values, train_costs, val_costs):
    print(f"λ={lam:<5}  Train={tr:.4f}  Val={va:.4f}")

# pick best lambda by min validation cost
best_idx = int(np.argmin(val_costs))
best_lambda = lambda_values[best_idx]
best_theta  = thetas[best_idx]
print(f"\nBest λ: {best_lambda}")


In [ ]:
# === STEP 6: Evaluation ===
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_curve, auc, precision_recall_curve
import matplotlib.pyplot as plt
import seaborn as sns

def predict_prob(X, theta):
    return sigmoid(X @ theta)

def predict_label(X, theta, threshold=0.5):
    return (predict_prob(X, theta) >= threshold).astype(int)

# Use best lambda/theta from tuning
theta_final = best_theta
y_prob = predict_prob(X_val, theta_final)
y_pred = (y_prob >= 0.5).astype(int)

acc  = accuracy_score(y_val, y_pred)
prec = precision_score(y_val, y_pred, zero_division=0)
rec  = recall_score(y_val, y_pred, zero_division=0)
f1   = f1_score(y_val, y_pred, zero_division=0)
print(f"Accuracy:  {acc:.3f}\nPrecision: {prec:.3f}\nRecall:    {rec:.3f}\nF1 Score:  {f1:.3f}")

cm = confusion_matrix(y_val, y_pred)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted'); plt.ylabel('Actual'); plt.title('Confusion Matrix'); plt.show()

fpr, tpr, _ = roc_curve(y_val, y_prob)
roc_auc = auc(fpr, tpr)
plt.plot(fpr, tpr, label=f"AUC = {roc_auc:.2f}")
plt.plot([0,1],[0,1],'--',alpha=0.6)
plt.xlabel('False Positive Rate'); plt.ylabel('True Positive Rate'); plt.title('ROC Curve'); plt.legend(); plt.show()

precisions, recalls, _ = precision_recall_curve(y_val, y_prob)
plt.plot(recalls, precisions)
plt.xlabel('Recall'); plt.ylabel('Precision'); plt.title('Precision–Recall Curve'); plt.show()
